In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('nominations.db')

In [3]:
#Checking Schema
schema = conn.execute("pragma table_info(nominations)")
schema.fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'charachter', 'text', 0, None, 0),
 (5, 'Won', 'integer', 0, None, 0),
 (6, 'ceremony_id', 'integer', 0, None, 0)]

In [4]:
first_ten = conn.execute("select * from nominations limit 10").fetchall()

for i in first_ten:
    print(i)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful ', "'Uxbal'", 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit ', "'Rooster Cogburn'", 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network ', "'Mark Zuckerberg'", 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech ", "'King George VI'", 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours ', "'Aron Ralston'", 0, 1)
(6, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter ', "'Dicky Eklund'", 1, 1)
(7, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone ", "'Teardrop'", 0, 1)
(8, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town ', "'James Coughlin'", 0, 1)
(9, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right ', "'Paul'", 0, 1)
(10, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech ", "'Lionel Logue'", 0, 1)


In [5]:
#Creating a ceremonies table that contains the list of tuples connecting ceremonial hosts corresponding to year 
#List of tuples
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

In [7]:
#Creating table

query = '''create table ceremonies(
id integer primary key,
Year integer,
Host text
);'''

f = conn.execute(query)

In [8]:
#Inserting years_host list into ceremonies table

insert_query = ''' insert into ceremonies (Year,Host) values (?,?)'''
conn.executemany(insert_query,years_hosts)


In [9]:
f = conn.execute("pragma table_info(ceremonies)").fetchall()
f

[(0, 'id', 'integer', 0, None, 1),
 (1, 'Year', 'integer', 0, None, 0),
 (2, 'Host', 'text', 0, None, 0)]

In [23]:
f = conn.execute('select * from ceremonies limit 10').fetchall()
f

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [24]:
f

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

# Fixes

Fixing the schema when the column names are incorrect.
Rename the table(with errors) to tempName and then recreate the original tableName and then insert the values into newly created table.


https://stackoverflow.com/questions/805363/how-do-i-rename-a-column-in-a-sqlite-database-table

        fix1 = '''alter table ceremonies rename to tmp_table'''
        conn.execute(fix1)

Removing the old tmp table:

        fix2 = "drop table tmp_table"
        conn.execute(fix2)

# Foreign Keys

        PRAGMA foreign_keys = ON;

The above query needs to be run every time you connect to a database where you'll be inserting foreign keys.

In [25]:
conn.execute('pragma foreign_keys = on')

In [43]:
#Creating an append listfrom old table to insert into new table.

query = '''SELECT ceremonies.id, nominations.category, nominations.nominee, nominations.movie, 
nominations.charachter, nominations.won
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
;'''


In [45]:
conn.execute("pragma table_info(nominations)").fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'charachter', 'text', 0, None, 0),
 (5, 'Won', 'integer', 0, None, 0),
 (6, 'ceremony_id', 'integer', 0, None, 0)]

In [27]:
join_list = conn.execute(query).fetchall()
for i in join_list[:10]:    
    print(i)

OperationalError: no such column: nominations.year

In [14]:
fix1 = '''alter table nominations rename to tmp_table2'''
conn.execute(fix1)

In [15]:
#Creating the new table with desired schema

create_table = '''create table nominations(
id integer primary key,
category text,
nominee text,
movie text,
charachter text,
Won integer,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id)
);
'''

In [16]:
conn.execute(create_table)

In [17]:
insert_query = 'insert into nominations (ceremony_id,category,nominee,movie,charachter,Won) values (?,?,?,?,?,?)'

In [18]:
conn.executemany(insert_query,join_list)

NameError: name 'join_list' is not defined

In [19]:
f = conn.execute('pragma table_info(nominations)').fetchall()
f

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'charachter', 'text', 0, None, 0),
 (5, 'Won', 'integer', 0, None, 0),
 (6, 'ceremony_id', 'integer', 0, None, 0)]

In [20]:
f = conn.execute('select * from nominations limit 10').fetchall()

In [21]:
#Drop tmp_table

fix1 = '''alter table actors rename to tmp_table'''
conn.execute(fix1)

In [ ]:
fix2 = "drop table movies_actors"
conn.execute(fix2)

# Many to Many relations through a join table

In [28]:
#Creating the tables

actors = '''create table actors(
id integer primary key,
actor text
)'''

conn.execute(actors)

In [29]:
movies = '''create table movies(
id integer primary key,
movie text
)
'''
conn.execute(movies)

OperationalError: table movies already exists

In [30]:
movies_actors = '''create table movies_actors(
id integer primary key,
movie_id integer references movies(id),
actor_id integer references actors(id)
);'''

conn.execute(movies_actors)

OperationalError: table movies_actors already exists

In [31]:
actor_sql= "select distinct nominations.nominee from nominations;"
actor_list = conn.execute(actor_sql).fetchall()
actor_list[:5]

[]

In [32]:
conn.executemany("insert into actors (actor) values (?);",actor_list)

In [33]:
movie_sql = "select distinct nominations.movie from nominations;"
movie_list = conn.execute(movie_sql).fetchall()
movie_list[:5]

[]

In [34]:
conn.executemany("insert into movies (movie) values(?)",movie_list)

In [35]:
query = "select * from movies"

f = conn.execute(query).fetchall()
f[:5]

[(1, 'Biutiful '),
 (2, 'True Grit '),
 (3, 'The Social Network '),
 (4, "The King's Speech "),
 (5, '127 Hours ')]

In [36]:
query = "select * from actors"

f = conn.execute(query).fetchall()
f[:5]

[]

In [37]:
joint_query = '''select * from actors
inner join nominations on actors.actor == nominations.nominee'''

In [38]:
f = conn.execute(joint_query)
for i in f.fetchall()[:10]:
    print(i)
f.fetchall()

[]

In [39]:
f = conn.execute("pragma table_info(movies_actors)").fetchall()
f

[(0, 'id', 'integer', 0, None, 1),
 (1, 'movie_id', 'integer', 0, None, 0),
 (2, 'actor_id', 'integer', 0, None, 0)]

In [40]:
query = "select * from movies_actors"

f = conn.execute(query).fetchall()
f[:5]

[]

In [41]:
query = '''select ceremonies.year, nominations.movie from nominations 
inner join ceremonies on 
nominations.ceremony_id == ceremonies.id 
where nominations.nominee == 'Natalie Portman';
'''

f = conn.execute(query)

portman_movies = f.fetchall()
print(portman_movies)

[]


In [42]:
query = '''SELECT * FROM movies
INNER JOIN movies_actors ON movies.id == movies_actors.movie_id'''

f = conn.execute(query)
f.fetchall()

[]